In [245]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import os
from IPython.core.display import display, HTML
import pickle

C:\Users\nekna\AppData\Local\Temp\ipykernel_35120\1480811194.py:6: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


# EDA

In [246]:
os.chdir('D:\\hse_workshop_classification\\')
train = pd.read_csv('data/raw/train.csv')
test = pd.read_csv('data/raw/test.csv')

In [247]:
display(HTML("<h3>TRAIN DATASET</h3>"))
display(HTML(train.head().to_html()))
display(HTML("<h3>TEST DATASET</h3>"))
display(HTML(test.head().to_html()))

,ID,Пол,Семья,Этнос,Национальность,Религия,Образование,Профессия,Вы работаете?,Выход на пенсию,Прекращение работы по болезни,Сахарный диабет,Гепатит,Онкология,Хроническое заболевание легких,Бронжиальная астма,Туберкулез легких,ВИЧ/СПИД,Регулярный прим лекарственных средств,Травмы за год,Переломы,Статус Курения,Возраст курения,Сигарет в день,Пассивное курение,Частота пасс кур,Алкоголь,Возраст алког,Время засыпания,Время пробуждения,Сон после обеда,"Спорт, клубы","Религия, клубы",ID_y,Артериальная гипертензия,ОНМК,"Стенокардия, ИБС, инфаркт миокарда",Сердечная недостаточность,Прочие заболевания сердца
0,54-102-358-02,М,в браке в настоящее время,европейская,Русские,Христианство,3 - средняя школа / закон.среднее / выше среднего,низкоквалифицированные работники,1,0,0,0,0,0,0,0,0,0,0,0,0,Курит,15.0,20.0,0,NaN,употребляю в настоящее время,18.0,22:00:00,06:00:00,0,0,0,54-102-358-02,0,0,0,0,0
1,54-103-101-01,Ж,в разводе,европейская,Русские,Христианство,5 - ВУЗ,дипломированные специалисты,0,0,0,1,0,0,0,0,0,0,1,0,1,Никогда не курил(а),NaN,NaN,0,NaN,никогда не употреблял,NaN,00:00:00,04:00:00,1,0,0,54-103-101-01,1,1,0,0,0
2,54-501-026-03,Ж,в браке в настоящее время,европейская,Русские,Христианство,5 - ВУЗ,дипломированные специалисты,0,0,0,0,0,0,0,0,0,0,1,0,0,Никогда не курил(а),NaN,NaN,1,1-2 раза в неделю,употребляю в настоящее время,17.0,23:00:00,07:00:00,0,0,0,54-501-026-03,0,0,0,0,0
3,54-501-094-02,М,в браке в настоящее время,европейская,Русские,Атеист / агностик,3 - средняя школа / закон.среднее / выше среднего,низкоквалифицированные работники,1,0,0,0,0,1,0,0,0,0,1,0,0,Бросил(а),12.0,10.0,1,3-6 раз в неделю,употребляю в настоящее время,13.0,23:00:00,07:00:00,0,0,0,54-501-094-02,1,0,0,0,0
4,54-503-022-01,Ж,в браке в настоящее время,европейская,Русские,Христианство,3 - средняя школа / закон.среднее / выше среднего,операторы и монтажники установок и машинного оборудования,0,0,1,1,1,0,0,0,0,0,1,0,1,Никогда не курил(а),NaN,NaN,1,не менее 1 раза в день,употребляю в настоящее время,16.0,23:00:00,06:00:00,0,0,0,54-503-022-01,1,0,1,1,0


,ID,Пол,Семья,Этнос,Национальность,Религия,Образование,Профессия,Вы работаете?,Выход на пенсию,Прекращение работы по болезни,Сахарный диабет,Гепатит,Онкология,Хроническое заболевание легких,Бронжиальная астма,Туберкулез легких,ВИЧ/СПИД,Регулярный прим лекарственных средств,Травмы за год,Переломы,Статус Курения,Возраст курения,Сигарет в день,Пассивное курение,Частота пасс кур,Алкоголь,Возраст алког,Время засыпания,Время пробуждения,Сон после обеда,"Спорт, клубы","Религия, клубы"
0,54-001-019-01,Ж,вдовец / вдова,европейская,Русские,Христианство,3 - средняя школа / закон.среднее / выше среднего,ремесленники и представители других отраслей промышленности,1,0,0,0,0,0,0,0,0,0,1,1,1,Никогда не курил(а),NaN,NaN,0,NaN,употребляю в настоящее время,23.0,21:30:00,05:00:00,0,1,1
1,54-002-133-01,Ж,в разводе,европейская,Русские,Христианство,5 - ВУЗ,дипломированные специалисты,0,1,0,0,0,0,0,0,0,0,1,0,0,Никогда не курил(а),NaN,NaN,0,NaN,употребляю в настоящее время,22.0,00:00:00,09:00:00,0,0,0
2,54-001-007-01,Ж,в браке в настоящее время,европейская,Русские,Христианство,5 - ВУЗ,дипломированные специалисты,0,1,0,0,0,0,0,0,0,0,1,0,0,Никогда не курил(а),NaN,NaN,0,NaN,употребляю в настоящее время,18.0,23:00:00,07:30:00,0,0,0
3,54-102-116-01,Ж,в браке в настоящее время,европейская,Русские,Христианство,4 - профессиональное училище,низкоквалифицированные работники,1,0,0,0,1,0,0,0,0,0,0,0,1,Никогда не курил(а),NaN,NaN,0,NaN,никогда не употреблял,NaN,22:00:00,06:00:00,0,0,0
4,54-502-005-02,М,в браке в настоящее время,европейская,Русские,Христианство,3 - средняя школа / закон.среднее / выше среднего,операторы и монтажники установок и машинного оборудования,0,1,0,0,0,0,0,0,0,0,1,1,1,Курит,14.0,20.0,0,NaN,употребляю в настоящее время,19.0,22:00:00,06:00:00,0,0,0


Как мы можем заметить, в столбцах возраст курения, сигарет в день и частота пассивного курения присутствуют пропуски. Избавимся от них следующим образом: Если человек никогда не курил, то возраст курения и сигарет в день приравниваем к нулю. Аналогично с пассивным курением.

In [248]:
(train['ID'] == train['ID_y']).all()

True

Также заметим, что id == id_y, поэтому мы можем просто отбросить этот столбец

In [249]:
def Preprocessing(df: pd.DataFrame) -> pd.DataFrame:
    if 'ID_y' in df.columns:
        df = df.drop('ID_y', axis=1)
    if 'ID' in df.columns:
        df = df.set_index('ID')
    #Избавляемся от пропусков
    df['Возраст курения'] = np.where(df['Статус Курения'] == 'Никогда не курил(а)', 0, df['Возраст курения'])
    df['Сигарет в день'] = np.where(df['Статус Курения'] == 'Никогда не курил(а)', 0, df['Сигарет в день'])
    df['Сигарет в день'] = np.where(df['Сигарет в день'].isna(), 0, df['Сигарет в день'])
    df['Частота пасс кур'] = np.where(df['Пассивное курение'] == 0, 'Ни разу в день', df['Частота пасс кур'])
    df['Возраст алког'] = np.where(df['Алкоголь'] == 'никогда не употреблял', 0, df['Возраст алког'])

    df = df.drop(df[df['Возраст алког'].isna()].index)
    df = df.drop(df[df['Пол'].isna()].index)
    df = df.drop(df[df['Частота пасс кур'].isna()].index)

    return df


In [250]:
train = Preprocessing(train)
test = test.set_index('ID')

Мы избавились от всех пропусков. Теперь уберем столбцы, которые мы будем предсказывать

In [251]:
clear_train = train.copy()
Target_columns = [
    'Артериальная гипертензия', 'ОНМК',
    'Стенокардия, ИБС, инфаркт миокарда',
    'Сердечная недостаточность', 
    'Прочие заболевания сердца'
]
target = clear_train[Target_columns]
clear_train = clear_train.drop(Target_columns, axis=1)
display(HTML(target.head().to_html()))
display(HTML(clear_train.head().to_html()))

,Артериальная гипертензия,ОНМК,"Стенокардия, ИБС, инфаркт миокарда",Сердечная недостаточность,Прочие заболевания сердца
ID,,,,,
54-102-358-02,0,0,0,0,0
54-103-101-01,1,1,0,0,0
54-501-026-03,0,0,0,0,0
54-501-094-02,1,0,0,0,0
54-503-022-01,1,0,1,1,0


,Пол,Семья,Этнос,Национальность,Религия,Образование,Профессия,Вы работаете?,Выход на пенсию,Прекращение работы по болезни,Сахарный диабет,Гепатит,Онкология,Хроническое заболевание легких,Бронжиальная астма,Туберкулез легких,ВИЧ/СПИД,Регулярный прим лекарственных средств,Травмы за год,Переломы,Статус Курения,Возраст курения,Сигарет в день,Пассивное курение,Частота пасс кур,Алкоголь,Возраст алког,Время засыпания,Время пробуждения,Сон после обеда,"Спорт, клубы","Религия, клубы"
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
54-102-358-02,М,в браке в настоящее время,европейская,Русские,Христианство,3 - средняя школа / закон.среднее / выше среднего,низкоквалифицированные работники,1,0,0,0,0,0,0,0,0,0,0,0,0,Курит,15.0,20.0,0,Ни разу в день,употребляю в настоящее время,18.0,22:00:00,06:00:00,0,0,0
54-103-101-01,Ж,в разводе,европейская,Русские,Христианство,5 - ВУЗ,дипломированные специалисты,0,0,0,1,0,0,0,0,0,0,1,0,1,Никогда не курил(а),0.0,0.0,0,Ни разу в день,никогда не употреблял,0.0,00:00:00,04:00:00,1,0,0
54-501-026-03,Ж,в браке в настоящее время,европейская,Русские,Христианство,5 - ВУЗ,дипломированные специалисты,0,0,0,0,0,0,0,0,0,0,1,0,0,Никогда не курил(а),0.0,0.0,1,1-2 раза в неделю,употребляю в настоящее время,17.0,23:00:00,07:00:00,0,0,0
54-501-094-02,М,в браке в настоящее время,европейская,Русские,Атеист / агностик,3 - средняя школа / закон.среднее / выше среднего,низкоквалифицированные работники,1,0,0,0,0,1,0,0,0,0,1,0,0,Бросил(а),12.0,10.0,1,3-6 раз в неделю,употребляю в настоящее время,13.0,23:00:00,07:00:00,0,0,0
54-503-022-01,Ж,в браке в настоящее время,европейская,Русские,Христианство,3 - средняя школа / закон.среднее / выше среднего,операторы и монтажники установок и машинного оборудования,0,0,1,1,1,0,0,0,0,0,1,0,1,Никогда не курил(а),0.0,0.0,1,не менее 1 раза в день,употребляю в настоящее время,16.0,23:00:00,06:00:00,0,0,0


Разделим признаки на несколько списков для удобства

In [252]:
clear_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 950 entries, 54-102-358-02 to 54-702-016-01
Data columns (total 32 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   Пол                                    950 non-null    object 
 1   Семья                                  950 non-null    object 
 2   Этнос                                  950 non-null    object 
 3   Национальность                         950 non-null    object 
 4   Религия                                950 non-null    object 
 5   Образование                            950 non-null    object 
 6   Профессия                              950 non-null    object 
 7   Вы работаете?                          950 non-null    int64  
 8   Выход на пенсию                        950 non-null    int64  
 9   Прекращение работы по болезни          950 non-null    int64  
 10  Сахарный диабет                        950 non-null    in

In [253]:
Categorial_columns = [
    'Пол', 'Семья', 'Этнос', 'Национальность', 'Религия', 'Образование', 
    'Профессия', 'Статус Курения', 'Частота пасс кур', 'Алкоголь',
    'Время засыпания', 'Время пробуждения'
]
Ohe_columns = [
    'Пол', 'Вы работаете?', 'Выход на пенсию', 'Прекращение работы по болезни', 'Сахарный диабет', 'Гепатит',
    'Онкология', 'Хроническое заболевание легких', 'Бронжиальная астма', 'Туберкулез легких ', 'ВИЧ/СПИД',
    'Регулярный прим лекарственных средств', 'Травмы за год', 'Переломы','Пассивное курение', 'Сон после обеда', 
    'Спорт, клубы', 'Религия, клубы'
]
Real_columns = ['Возраст курения', 'Сигарет в день', 'Возраст алког']
Education_col = 'Образование'
clear_train[Education_col] = clear_train[Education_col].str.slice(0,1).astype(np.int8).values
test[Education_col] = test[Education_col].str.slice(0,1).astype(np.int8).values


Ohe_int_columns = [Ohe_columns[i] for i in range(len(Ohe_columns)) if type(clear_train[Ohe_columns[i]][0]) == np.int64]
clear_train

,Пол,Семья,Этнос,Национальность,Религия,Образование,Профессия,Вы работаете?,Выход на пенсию,Прекращение работы по болезни,...,Сигарет в день,Пассивное курение,Частота пасс кур,Алкоголь,Возраст алког,Время засыпания,Время пробуждения,Сон после обеда,"Спорт, клубы","Религия, клубы"
ID,,,,,,,,,,,,,,,,,,,,,
54-102-358-02,М,в браке в настоящее время,европейская,Русские,Христианство,3,низкоквалифицированные работники,1,0,0,...,20.0,0,Ни разу в день,употребляю в настоящее время,18.0,22:00:00,06:00:00,0,0,0
54-103-101-01,Ж,в разводе,европейская,Русские,Христианство,5,дипломированные специалисты,0,0,0,...,0.0,0,Ни разу в день,никогда не употреблял,0.0,00:00:00,04:00:00,1,0,0
54-501-026-03,Ж,в браке в настоящее время,европейская,Русские,Христианство,5,дипломированные специалисты,0,0,0,...,0.0,1,1-2 раза в неделю,употребляю в настоящее время,17.0,23:00:00,07:00:00,0,0,0
54-501-094-02,М,в браке в настоящее время,европейская,Русские,Атеист / агностик,3,низкоквалифицированные работники,1,0,0,...,10.0,1,3-6 раз в неделю,употребляю в настоящее время,13.0,23:00:00,07:00:00,0,0,0
54-503-022-01,Ж,в браке в настоящее время,европейская,Русские,Христианство,3,операторы и монтажники установок и машинного о...,0,0,1,...,0.0,1,не менее 1 раза в день,употребляю в настоящее время,16.0,23:00:00,06:00:00,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54-103-006-01,М,в браке в настоящее время,европейская,Русские,Христианство,4,операторы и монтажники установок и машинного о...,1,0,0,...,20.0,0,Ни разу в день,употребляю в настоящее время,18.0,22:00:00,06:00:00,0,0,0
54-701-039-01,Ж,вдовец / вдова,европейская,Русские,Христианство,3,низкоквалифицированные работники,1,0,0,...,10.0,0,Ни разу в день,ранее употреблял,26.0,21:30:00,07:15:00,0,0,0
54-002-014-01,Ж,в браке в настоящее время,европейская,Русские,Христианство,3,служащие,0,1,0,...,10.0,0,Ни разу в день,употребляю в настоящее время,20.0,22:30:00,08:00:00,1,0,0


Преобразование типов

In [254]:
clear_train[Ohe_int_columns] = clear_train[Ohe_int_columns].astype(np.int8)
clear_train[Categorial_columns] = clear_train[Categorial_columns].astype('category')
clear_train[Real_columns] = clear_train[Real_columns].astype(np.float32)

In [255]:
clear_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 950 entries, 54-102-358-02 to 54-702-016-01
Data columns (total 32 columns):
 #   Column                                 Non-Null Count  Dtype   
---  ------                                 --------------  -----   
 0   Пол                                    950 non-null    category
 1   Семья                                  950 non-null    category
 2   Этнос                                  950 non-null    category
 3   Национальность                         950 non-null    category
 4   Религия                                950 non-null    category
 5   Образование                            950 non-null    category
 6   Профессия                              950 non-null    category
 7   Вы работаете?                          950 non-null    int8    
 8   Выход на пенсию                        950 non-null    int8    
 9   Прекращение работы по болезни          950 non-null    int8    
 10  Сахарный диабет                        950 no

In [256]:
display(HTML(clear_train.head().to_html()))

,Пол,Семья,Этнос,Национальность,Религия,Образование,Профессия,Вы работаете?,Выход на пенсию,Прекращение работы по болезни,Сахарный диабет,Гепатит,Онкология,Хроническое заболевание легких,Бронжиальная астма,Туберкулез легких,ВИЧ/СПИД,Регулярный прим лекарственных средств,Травмы за год,Переломы,Статус Курения,Возраст курения,Сигарет в день,Пассивное курение,Частота пасс кур,Алкоголь,Возраст алког,Время засыпания,Время пробуждения,Сон после обеда,"Спорт, клубы","Религия, клубы"
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
54-102-358-02,М,в браке в настоящее время,европейская,Русские,Христианство,3,низкоквалифицированные работники,1,0,0,0,0,0,0,0,0,0,0,0,0,Курит,15.0,20.0,0,Ни разу в день,употребляю в настоящее время,18.0,22:00:00,06:00:00,0,0,0
54-103-101-01,Ж,в разводе,европейская,Русские,Христианство,5,дипломированные специалисты,0,0,0,1,0,0,0,0,0,0,1,0,1,Никогда не курил(а),0.0,0.0,0,Ни разу в день,никогда не употреблял,0.0,00:00:00,04:00:00,1,0,0
54-501-026-03,Ж,в браке в настоящее время,европейская,Русские,Христианство,5,дипломированные специалисты,0,0,0,0,0,0,0,0,0,0,1,0,0,Никогда не курил(а),0.0,0.0,1,1-2 раза в неделю,употребляю в настоящее время,17.0,23:00:00,07:00:00,0,0,0
54-501-094-02,М,в браке в настоящее время,европейская,Русские,Атеист / агностик,3,низкоквалифицированные работники,1,0,0,0,0,1,0,0,0,0,1,0,0,Бросил(а),12.0,10.0,1,3-6 раз в неделю,употребляю в настоящее время,13.0,23:00:00,07:00:00,0,0,0
54-503-022-01,Ж,в браке в настоящее время,европейская,Русские,Христианство,3,операторы и монтажники установок и машинного оборудования,0,0,1,1,1,0,0,0,0,0,1,0,1,Никогда не курил(а),0.0,0.0,1,не менее 1 раза в день,употребляю в настоящее время,16.0,23:00:00,06:00:00,0,0,0


In [257]:
clear_train.to_pickle(path='data/processed/train.pkl')
test.to_pickle(path='data/processed/test.pkl')

## Modeling

Будем использовать метрику recall, потому что нам необходимо жестко контролировать количество тех больных, которые попали в группу здоровых. Так как recall = $\frac{True\ positive}{True\ positive+False\ negative}$, то эта метрика хорошо отражает необходимое нам отношение

## RidgeClassifier from sklearn

In [258]:
from sklearn.metrics import recall_score
os.chdir('D:\\hse_workshop_classification\\')
#print(os.getcwd())
from sklearn.svm import *
from sklearn.model_selection import *
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import RidgeClassifier
from sklearn.pipeline import *
from src.features import features

import src.config as cfg

train = pd.read_pickle('data/processed/train.pkl')

train[cfg.CAT_COLS] = train[cfg.CAT_COLS].astype('object')

train = features.add_early_wakeup(train)
encod = LabelEncoder()
for i in cfg.OHE_COLS:
        train[i] = encod.fit_transform(train[i])
for i in cfg.REAL_COLS:
        train[i] = encod.fit_transform(train[i])
for i in cfg.CAT_COLS:
        train[i] = encod.fit_transform(train[i])
train_x, test_x, train_y, test_y = train_test_split(train, target, test_size=0.2, random_state=1807)

train_x


,Пол,Семья,Этнос,Национальность,Религия,Образование,Профессия,Вы работаете?,Выход на пенсию,Прекращение работы по болезни,...,Пассивное курение,Частота пасс кур,Алкоголь,Возраст алког,Время засыпания,Время пробуждения,Сон после обеда,"Спорт, клубы","Религия, клубы",Жаворонок
ID,,,,,,,,,,,,,,,,,,,,,
54-103-071-01,0,2,1,12,3,1,4,0,1,0,...,1,0,2,12,18,8,0,0,1,1
54-701-041-01,0,0,1,12,3,2,2,1,0,0,...,0,4,2,10,15,16,0,0,0,1
54-101-032-01,0,0,1,12,3,2,0,0,1,0,...,0,4,2,12,18,11,0,0,0,1
54-501-062-01,0,0,1,12,3,3,2,0,1,0,...,1,3,2,27,18,4,0,0,0,1
54-501-045-02,1,3,1,12,3,1,4,1,0,0,...,1,2,2,4,15,20,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54-002-163-02,1,3,1,12,3,3,8,1,0,0,...,0,4,2,9,18,20,0,0,0,1
54-002-139-01,0,4,1,12,3,3,9,1,0,0,...,0,4,2,10,18,20,0,0,0,1
54-103-092-01,1,0,1,12,3,3,6,1,0,0,...,1,0,2,13,18,20,0,0,0,1


In [259]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score


ridge_class = RidgeClassifier()
ridge_class.fit(train_x, train_y)
predicted_rc = ridge_class.predict(test_x)

print(f"recall micro  {recall_score(test_y, predicted_rc, average='micro')}\n",
      f"recall samples {recall_score(test_y, predicted_rc, average='samples')}\n"
      f"accuracy {accuracy_score(test_y, predicted_rc)}\n",
      f"roc auc {roc_auc_score(test_y, predicted_rc)}"
     )



recall micro  0.4722222222222222
 recall samples 0.2798245614035087
accuracy 0.5421052631578948
 roc auc 0.5454185071922074


C:\Users\nekna\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Как мы видим, модель работает не самым лучшим образом. Recall более 47% по всей выборке и около 28% по каждому классу в отдельности

## CatBoost

In [260]:
from catboost import Pool, CatBoostClassifier

#train_data = Pool(train_x, train_y)

catboost_model = CatBoostClassifier(iterations=1000, loss_function='MultiLogloss', 
                                    eval_metric='MultiLogloss', learning_rate=0.03, 
                                    bootstrap_type='Bayesian', boost_from_average=False, 
                                    leaf_estimation_iterations=1, leaf_estimation_method='Gradient')
catboost_model.fit(train_x, train_y)

predicted_cb = catboost_model.predict(test_x)

0:	learn: 0.6892230	total: 7.09ms	remaining: 7.08s
1:	learn: 0.6853279	total: 10.8ms	remaining: 5.39s
2:	learn: 0.6813608	total: 15.5ms	remaining: 5.15s
3:	learn: 0.6775723	total: 19.3ms	remaining: 4.81s
4:	learn: 0.6739368	total: 23.1ms	remaining: 4.59s
5:	learn: 0.6701189	total: 26.5ms	remaining: 4.38s
6:	learn: 0.6666172	total: 30.1ms	remaining: 4.28s
7:	learn: 0.6631980	total: 33.3ms	remaining: 4.13s
8:	learn: 0.6597447	total: 36.5ms	remaining: 4.02s
9:	learn: 0.6563233	total: 39.3ms	remaining: 3.89s
10:	learn: 0.6529497	total: 42.7ms	remaining: 3.84s
11:	learn: 0.6495530	total: 45.4ms	remaining: 3.74s
12:	learn: 0.6461464	total: 48.5ms	remaining: 3.68s
13:	learn: 0.6427321	total: 51.4ms	remaining: 3.62s
14:	learn: 0.6396417	total: 54.5ms	remaining: 3.58s
15:	learn: 0.6362751	total: 57.5ms	remaining: 3.54s
16:	learn: 0.6331621	total: 60.5ms	remaining: 3.5s
17:	learn: 0.6300091	total: 61.5ms	remaining: 3.36s
18:	learn: 0.6267866	total: 64.2ms	remaining: 3.32s
19:	learn: 0.6235551	to

In [261]:
print(f"Recall micro {recall_score(test_y, predicted_cb, average='micro')}\n",
    f"Recall samples {recall_score(test_y, predicted_cb, average='samples')}\n",
    f"Accuracy {accuracy_score(test_y, predicted_cb)}\n",
    f"roc auc {roc_auc_score(test_y, predicted_cb)}"
    )

Recall micro 0.4652777777777778
 Recall samples 0.2745614035087719
 Accuracy 0.5473684210526316
 roc auc 0.546018801676294


C:\Users\nekna\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Как мы видим, у обеих моделей rocauc больше 0.5, следовательно модели выдают не случайные числа

Завернем обучение моделей в пайплайн и протестируем на recall scores

In [262]:
import category_encoders as ce
from sklearn.compose import *
from sklearn.impute import *
from sklearn.multioutput import *
from sklearn.preprocessing import *

real_pipe = Pipeline([
        ('imputer', SimpleImputer()),
        ('scaler', StandardScaler())
    ]
    )
cat_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='constant')),
    ('ohe', OneHotEncoder(handle_unknown='ignore', sparse=False))
])
preprocess_pipe = ColumnTransformer(transformers=[
    ('real_cols', real_pipe, cfg.REAL_COLS),
    ('cat_cols', cat_pipe, cfg.CAT_COLS),
    ('woe_cat_cols', ce.WOEEncoder(), cfg.CAT_COLS),
    ('ohe_cols', 'passthrough', cfg.OHE_COLS)
]
)


def pipelining(metric:str):
    ridge_classifier = RidgeClassifier()
    ridge_pipe = Pipeline([
        ('preprocess', preprocess_pipe),
        ('model', ridge_classifier)
    ]
    )
    catboost_classifier = CatBoostClassifier(iterations=1000, loss_function='MultiLogloss', 
                                            eval_metric='MultiLogloss', learning_rate=0.03, 
                                            bootstrap_type='Bayesian', boost_from_average=False, 
                                            leaf_estimation_iterations=1, leaf_estimation_method='Gradient')
    catboost_pipe = Pipeline([
        ('preprocess', preprocess_pipe),
        ('model', catboost_classifier)
    ]
    )
    multiout_model_pipe_ridge = MultiOutputClassifier(ridge_pipe, n_jobs=3)
    ridge_scores = cross_val_score(
        estimator=multiout_model_pipe_ridge,
        X=train_x,
        y=train_y,
        scoring=metric,
        cv=5,
        n_jobs=1
    )
    multiout_model_pipe_catboost = MultiOutputClassifier(ridge_pipe, n_jobs=3)
    catboost_scores = cross_val_score(
        estimator=multiout_model_pipe_catboost,
        X=train_x,
        y=train_y,
        scoring=metric,
        cv=5,
        n_jobs=1
    )
    return ridge_scores, catboost_scores

for metric in ['recall_samples', 'recall_micro']:
    ridge_scores, catboost_scores = pipelining(metric)
    print(f'RidgeClassifier {metric} scores are {ridge_scores}\nCatBoost classifier {metric} scores are {catboost_scores}\n\n')

C:\Users\nekna\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\nekna\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\nekna\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall is ill-de

RidgeClassifier recall_samples scores are [0.22368421 0.25767544 0.26041667 0.2379386  0.20723684]
CatBoost classifier recall_samples scores are [0.22368421 0.25767544 0.26041667 0.2379386  0.20723684]


RidgeClassifier recall_micro scores are [0.36090226 0.37037037 0.44827586 0.40601504 0.36065574]
CatBoost classifier recall_micro scores are [0.36090226 0.37037037 0.44827586 0.40601504 0.36065574]




Как мы видим, средний recall scores у Ridge Classifier = 0.39, средний recall scores у CatBoost classifier = 0.39.